In [10]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itools
import scipy as sc



In [16]:
# A = np.array([[1.0189, 0.9051, 0.0022],
#     [0.8223, 1.0774, 0.1756],
#     [0, 0, -1.0000]])

A = np.array([[1, 1],
             [0, 1]])

# print('A=', A)
# print(A.shape)

# B = np.array([[0, 1],
#             [0, 1],
#             [1, 1]])

B = np.array([[0],
             [1]])

# print('B=', B)


n = np.shape(A)[0]
m = np.shape(B)[1]
nu = (n+m)*(n+m+1)/2

M = 20*np.diag(np.ones(n))
# print('M=', M)

R = 0.3*np.diag(np.ones(m))

a_c = 80

a_u = 20

T_e = 10

T = 1


hh = 0.5
step = T/hh

Wc = np.random.rand(int(nu),1)
Wu = np.random.rand(n,m)


# x = np.array([[1, 0.1, -0.5]]).T
x = np.array([[1, 0]]).T
u = np.matmul(Wu.T,x)
U = np.vstack((x,u))

Wu_vec = Wu.reshape((n*m,1))
p = np.matmul(x.T, np.matmul(M, x)) + np.matmul(u.T, np.matmul((R), u))


y = np.vstack((x,Wc,Wu_vec,p))
tspan = 12


In [17]:
# Q-Function Creation #

def Qcreate(Wc, n=n, m=m, nu=nu):

    Q = np.zeros((n+m, n+m))
    idx = np.triu_indices(n+m)
    Q[idx] = Wc.reshape((int(nu),))
    Q = Q + Q.T
    

    Qxx = Q[0:n,0:n]
    Quu = Q[n:,n:]
    Qxu = Q[0:n,n:n+m]
    
    return Q, Qxx, Quu, Qxu




In [18]:
# Euler's Method #

def euler_state(x, u, A_mat = A, B_mat = B, h = hh):
    states_sys = np.matmul(A_mat,x) + np.matmul(B_mat,u)
    x_n = x + h*states_sys
    
    return x_n

def euler_critic(U, p, p_p, U_p, Wc, Q_mat, h = hh, cs = a_c):
    
    current_est = np.matmul(U.T, np.matmul(Q_mat, U))
    past_est = np.matmul(U_p.T, np.matmul(Q_mat, U_p))
    integral_error = p - p_p
    
    ec = 0.5*(current_est - past_est + integral_error)
    basis_idx = list(itools.combinations_with_replacement(range(n+m),2))
    
    phil = []
    phil_p = []
    for i,j in basis_idx:
        phil.append(U[i]*U[j])
        phil_p.append(U_p[i]*U_p[j])
        
    phi = np.array(phil).reshape(len(basis_idx))
    phi_p = np.array(phil_p).reshape(len(basis_idx))
        
        
    sigma = phi-phi_p
    critic_sys = -1*cs*(sigma/(1+np.dot(sigma,sigma))**2)*ec
    Wc_n = Wc + h*critic_sys.T
    return Wc_n

def euler_actor(x, Wa, Q_uu, Q_ux, h = hh, a_s = a_u):
    ea = np.matmul(Wa.T, x) + np.matmul(np.linalg.inv(Q_uu),np.matmul(Q_ux,x))
    actor_sys = -1*a_s*np.matmul(x,ea.T)
    Wa_n = Wa + h*actor_sys
    return Wa_n
    



In [19]:
U_series = []
x_series = []
u_series = []
Wc_series = []
Wu_series = []
p_series = []



U_series.append(U)
x_series.append(x)
u_series.append(u)
Wc_series.append(Wc) 
Wu_series.append(Wu)
p_series.append(p)


for t in np.arange(0,100,0.001):
#     print('t=',t)
    x_n = euler_state(x_series[-1], u_series[-1])
    
    step = int(step)
#     print('Wc_series=',Wc_series)
    Q, Qxx, Quu, Qxu = Qcreate(Wc_series[-1])
    Qux = Qxu.T
    
    
    
    if len(p_series) < step:
        st = -1
#         p_pp = p_series[-step]
#         u_pp = u_series[-step]
#         x_pp = x_series[-step]
        
    else:
        st = -step
#         p_pp = p_series[-1]
#         u_pp = u_series[-1]
#         x_pp = x_series[-1]
    
    Wc_n = euler_critic(U_series[-1], p_series[-1], p_series[st], U_series[st], Wc_series[-1], Q)
    Wu_n = euler_actor(x, Wu_series[-1], Quu, Qux)
    
#     print('Wc_n = ', Wc_n)
    
    u_new = np.matmul(Wu_n.T,x_n)
    
    if t < T_e:
        u_n = u_new + 0.2*np.exp(-0.0000001*t)*(np.sin(4*t**2)**2*np.cos(t**2)+np.sin(2**t)**2*np.cos(0.1*t)+np.sin(-1.2*t)**2*np.cos(0.5*t)+np.sin(t)**5+np.sin(1.12*t)**2+np.cos(2.4*t)*np.sin(2.4*t)**3)
    else:
        u_n = u_new
        
        
    p_past = np.matmul(x_series[-1].T, np.matmul(M, x_series[-1])) + np.matmul(u_series[-1].T, np.matmul((R), u_series[-1]))
    p_now = np.matmul(x_n.T, np.matmul(M, x_n)) + np.matmul(u_n.T, np.matmul((R), u_n))
    
    p_n = 0.5*hh*(p_past + p_now)
    
    x_series.append(x_n)
    Wc_series.append(Wc_n)
    Wu_series.append(Wu_n)
    u_series.append(u_n)
    
    
    U_n = np.vstack((x_n,u_n))
    U_series.append(U_n)
    p_series.append(p_n)

<ipython-input-18-1e08942f8698>:29: RuntimeWarning: overflow encountered in double_scalars
  critic_sys = -1*cs*(sigma/(1+np.dot(sigma,sigma))**2)*ec
<ipython-input-19-628fe953d10d>:55: RuntimeWarning: overflow encountered in matmul
  p_now = np.matmul(x_n.T, np.matmul(M, x_n)) + np.matmul(u_n.T, np.matmul((R), u_n))
<ipython-input-18-1e08942f8698>:11: RuntimeWarning: overflow encountered in matmul
  current_est = np.matmul(U.T, np.matmul(Q_mat, U))
<ipython-input-18-1e08942f8698>:21: RuntimeWarning: overflow encountered in multiply
  phil.append(U[i]*U[j])
<ipython-input-18-1e08942f8698>:29: RuntimeWarning: invalid value encountered in true_divide
  critic_sys = -1*cs*(sigma/(1+np.dot(sigma,sigma))**2)*ec
<ipython-input-18-1e08942f8698>:29: RuntimeWarning: invalid value encountered in multiply
  critic_sys = -1*cs*(sigma/(1+np.dot(sigma,sigma))**2)*ec
<ipython-input-19-628fe953d10d>:54: RuntimeWarning: overflow encountered in matmul
  p_past = np.matmul(x_series[-1].T, np.matmul(M, x_